> You can open this notebook in Colab by clicking the Colab icon. Colab provides GPU for free. You can also run this notebook locally by installing the dependencies listed in `requirements.txt`.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osbm/unet_explainer/blob/main/tutorial-part1.ipynb)

In [ ]:
!pip install git+https://github.com/osbm/unet_explainer.git

In [ ]:
from unet_pytorch import ProstateDataset, print_model_info, predict, set_seed
from unet_pytorch import plot_predictions

from torch.utils.data import DataLoader
import torch
from torch import nn
from monai.networks.nets import UNet
import albumentations as A
from albumentations.pytorch import ToTensorV2

import os

In [ ]:
if not os.path.exists('data'):
    !wget -q https://huggingface.co/datasets/osbm/unet-explainer-data/resolve/main/data.zip
    !unzip -q data.zip

In [ ]:
!wget -q https://huggingface.co/datasets/osbm/unet-explainer-data/resolve/main/best_model.pth

In [ ]:
set_seed(42)

In [ ]:
image_size = 256
valid_transforms = A.Compose([
    A.Resize(height=image_size, width=image_size),
    ToTensorV2(),
])
test_ds = ProstateDataset(folder='data/test', transform=valid_transforms)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)

In [ ]:
model = UNet(
    spatial_dims=2, # 2d image
    in_channels=1,  # we only used  T2 weighed MRI images
    out_channels=3, # 3 labels
    channels=[16, 32, 64, 128, 256, 512],
    strides=(2, 2, 2, 2, 2), # CNN strides
    num_res_units=4, # residual connections
    dropout=0.15, # dropout rate
)
print_model_info(model)

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
x, y, y_hat = predict(model, test_loader=test_loader, device=device, final_activation="softmax")

In [ ]:
plot_predictions(x, y, y_hat, num_examples_to_plot=5)